### 1. Query all structures containing desired elements from pymatgen.
#### In this example, I am pulling structures containing Ti and O.
For more information go to: https://pymatgen.org/

In [ ]:
import os
from pymatgen.ext.matproj import MPRester
import itertools
import json
import yaml

# Replace "YourID" with API generated from here: https://materialsproject.org/open
MPR = MPRester("YourID")

# All bulk structures will be stored in the directory 'structures'
if not os.path.isdir('structures'):
    os.mkdir('structures')

# Set a band_gap range to obtain photocatalysts
band_gap_min = 0.1
band_gap_max = 5.0

all_results = dict()
band_gaps = dict()

for metal in ['Ti']:
    traget_element = ['O',metal]
    properties = ['formula', 'formation_energy_per_atom','material_id',
                  'unit_cell_formula','elements','final_structure','band_gap','spacegroup']
    elements = ['O', 'Ti']
    results = []
    for i in range(len(elements)):
        for combo in itertools.combinations(elements, i + 1):
            combo = list(set(list(combo) + [metal, 'O']))
            try:
                results.extend(MPR.query({"elements": {"$all": combo}, "nelements": i+1}, properties=properties))
            except:
                pass
    print('Query finished')
    structures = {}
    band_gaps[metal] = []

    i = 0
    element_results = {}
    screened_results = []

    for material in results:
        band_gap = material['band_gap']
        if band_gap >= band_gap_min and band_gap <= band_gap_max:
            include = True
        else:
            include = False
        element = material['elements']
        for e in element:
            if e in traget_element:
                if e in element_results:
                    element_results[e] += 1
                else:
                    element_results[e] = 1
        if include == True:
            i += 1
            formula = ''
            for elem, num in sorted(material['formula'].items()):
                formula += str(elem) + str(int(num))
            name = formula + '_' + str(int(material['spacegroup']['number']))
            E = material['formation_energy_per_atom']
            name += '_' + str(band_gap)
            structures[name] = material['final_structure']
            band_gaps[metal].append(material['band_gap'])
    all_results[metal] = structures
    for name, structure in structures.items():
        structure.to('cif', 'structures/' + name + '.cif')
    print(metal)
    print(len(all_results[metal]))
 

### 2. Make surfaces from bulk structures.

In [ ]:
import os
from pymatgen.core.surface import generate_all_slabs
from pymatgen.core.structure import Structure
from pymatgen.io.ase import AseAtomsAdaptor as trans
from pymatgen.analysis.adsorption import AdsorbateSiteFinder

from pymatgen import Molecule
import numpy as np
from ase.build import molecule
from ase.constraints import FixAtoms
from ase.visualize import view

if not os.path.isdir('surfaces'):
    os.mkdir('surfaces')

slabs_result = dict() # key: index of the structure, val: all slabs of the structure
index = 0 
for struct_file in os.listdir('structures'):
    struct = Structure.from_file(os.path.join('structures', struct_file))
    all_slabs = generate_all_slabs(struct, 1,  8, 10, repair=True)
    slabs_result[index] = all_slabs
    index += 1    
    

In [ ]:
def filter_surface(atoms):
    d = []
    indices = []
    for atom in atoms:
        if atom.symbol == 'N':
            o_inds = [a.index for a in atoms if a.symbol != 'N']
            dists = atoms.get_distances(atom.index, o_inds)
            d.append(dists)
            indices.append(atom.index)
    mean_dists = [np.mean(a) for a in d]
    closest_index = mean_dists.index(min(mean_dists)) # minimum distance to N
    dists = d[closest_index]
    if not min(dists) < 2:
        return False
    elif min(dists) < 0.9:
        return False
    syms = []
    d=[]
    for i, dist in enumerate(dists):
        if dist <= 2:
            syms.append(atoms[o_inds[i]].symbol)
            d.append(dist)
    if len(syms) == 1 and syms[0] == 'O':
        return False
    if syms[d.index(min(d))] == 'O':
        return False
    return True

# We want to find active sites for N2
n2  = Molecule("NN",-1*molecule('N2').positions)
count = 0
index = len(slabs_result)

for i in range(index):
    struct_name = os.listdir('structures')[i].split('.')[0]
    surfs = slabs_result[i]
    print(i)
    for surf in surfs:
        if len(surf) > 50:
            continue
        count += 1
        for site_type, height in zip(['ontop', 'bridge','hollow'], [1.9, 1.7,1.7]):
            site_finder = AdsorbateSiteFinder(surf)
            ads_structures = site_finder.generate_adsorption_structures(n2,find_args= {'distance':height,'positions':[site_type]})
            for number, ads_structure in enumerate(ads_structures):
                if len(ads_structure) > 50: # number of unique surface
                    continue
                atoms = trans.get_atoms(ads_structure)
                if not filter_surface(atoms):
                    continue
                constraints = FixAtoms(mask=[a.symbol != 'N' for a in atoms])
                atoms.set_constraint(c)
                millers = ''.join([str(a) for a in surf.miller_index])    
                file_name = 'surfaces/' + struct_name + '_'\
                            +  millers + 'wN2_{}_{}.traj'.format(site_type, number)
                atoms.write(file_name)
                for atom in atoms:
                    if atom.symbol == 'N':
                        atom.symbol = 'O' # change N to O at the site
                file_name = 'surfaces/' + struct_name + '_'\
                            +  millers + 'wO2_{}_{}.traj'.format(site_type, number)
                atoms.write(file_name)
        atoms = trans.get_atoms(surf)
        constraints = FixAtoms(mask=[a.symbol != 'N' for a in atoms])
        atoms.set_constraint(c)
        millers = ''.join([str(a) for a in surf.miller_index])
        atoms.write('surfaces/' + struct_name + '_' +  millers + '.traj')
